## Testboards

In [1]:
#Startboard
board1 = [[9, 9],
         [[0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0]]]
#Phase 1
board2 = [[7, 6],
         [[0, 0, 0, 0, 2, 0, 0, 0],
          [0, 1, 2, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 1, 2, 0, 0]]]
#Phase 2, keine Mühle
board3 = [[0, 0],
         [[1, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 1, 0, 0, 0, 1, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Phase 2, Mühle
board4 = [[0, 0],
         [[1, 1, 1, 2, 0, 1, 0, 0],
          [2, 0, 1, 0, 0, 0, 1, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]
#Phase 3 für Spieler 1
board5 = [[0, 0],
         [[0, 0, 1, 2, 0, 1, 0, 0],
          [2, 0, 0, 0, 0, 0, 0, 0],
          [0, 2, 0, 0, 1, 2, 0, 0]]]

## Spielphasen

Prüft in welcher Phase sich der Spieler befindet

In [2]:
def get_player_phase(pieces, player):
    [remaining, board] = pieces
    if remaining[player-1] >= 1: # Es wurden noch nicht alle 9 Steine gesetzt
            return 1
    elif count_player_pieces(pieces, player) == 3: # nur noch drei Steine auf dem Spielfeld
        return 3
    else: # sonst
        return 2

Berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 1 (Setzphase) ist

In [3]:
def possible_positions_phase_one(pieces, player):
    return empty_positions(pieces)

Berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 2 (Zugphase) ist

In [4]:
def possible_positions_phase_two(pieces, player):
    collected_piece = (1,2) #ToDo: Implementieren
    neighboring = set(neighboring_positions(collected_piece))
    empty = set(empty_positions(pieces))
    possible_positions = neighboring.intersection(empty)
    #ToDo: Was, wenn kein Stein bewegt werden kann?
    return [possible_positions]

Berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 3 (Springphase) ist

In [5]:
def possible_positions_phase_three(pieces, player):
    return empty_positions(pieces)

Berechnet in welcher Phase sich der Spieler befindet und dementsprechen, wo er als nächstes klicken kann
Kombiniert also die Funktionen oben

In [1]:
def next_positions(pieces, player):
    phase = get_player_phase(pieces, player)
    if phase == 1:
        return possible_positions_phase_one(pieces, player)
    elif phase == 2:
        return possible_positions_phase_two(pieces, player)
    else:
        return possible_positions_phase_three(pieces, player)

## Mühlen

Vor dem Zug werden die Mühlen gezählt, nach dem Zug werden die Mühlen gezählt. Falls Nachher > Vorher, wurde eine Mühle gebildet. ToDo: Funktioniert so nicht -> Zwickmühle

In [7]:
def count_muehlen(pieces, player):
    if count_player_pieces(pieces, player) < 3:
        return 0
    #ToDo: Implementieren
    return

## Hilfsfunktionen

In [8]:
def count_player_pieces(pieces, player):
    [_, board] = pieces
    return [position for ring in board for position in ring].count(player)

In [9]:
def empty_positions(pieces):
    [_, board] = pieces
    return [(ring, cell) for ring in range(3)
                        for cell in range(8)
                        if  board[ring][cell] == 0 
           ]

In [11]:
# position = (ring, cell)
def neighboring_positions(position):
    ring, cell = position
    if cell == 0:
        left_neighbor = (ring, 7)
    else:
        left_neighbor = (ring, cell - 1)
    if cell == 7:
        right_neighbor = (ring, 0)
    else:
        right_neighbor = (ring, cell + 1)
        
    positions = [left_neighbor, right_neighbor]
        
    if cell in (1, 3, 5, 7):
        if ring in (0, 2) :
            positions.append((1, cell))
        else:
            positions.append((ring - 1, cell))
            positions.append((ring + 1, cell))
    return positions